## Chess AI
construct $f(p)$ as a 3 layer deep 2048 units wide artificial neural network\
for each move, $f(p) = \max\limits_{p\rightarrow p_0} - f(p_0)$\

In [1]:
import os
import chess
import chess.pgn
import time
import math
from tqdm import tqdm
import random
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
mp.set_start_method("spawn")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LAMBDA = 10.0
EPOCHS = 10
KAPPA = 1.0

### Prepare dataset
1. Players will choose an optimal or near-optimal move. This means that for two position in succession 
$p \rightarrow q$ observed in the game, we will have $f(p) = -f(q)$
2. For the same reason above, going from $p$ not to $q$, but to a random position $r$, we must have $f(r) > f(q)$ because the random position is better for the next player and worse for the player that made the move.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dimitrioskourtikakis/gm-games-chesscom")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/dimitrioskourtikakis/gm-games-chesscom/versions/1


In [4]:
import pandas
chessGames = pandas.read_csv(path+"/GM_games_dataset.csv", chunksize=1000)
# copy the "pgn" column to a pgn file
for i, chunk in enumerate(tqdm(chessGames)):
    with open(f"./data/chessGame{i}.pgn", "w") as f:
        for pgn in chunk['pgn']:
            f.write(pgn + "\n\n")

4812it [02:23, 33.48it/s]


In [2]:
piece_to_index = {
    'P': 0,  'N': 1,  'B': 2,  'R': 3,  'Q': 4,  'K': 5,  # White pieces
    'p': 6,  'n': 7,  'b': 8,  'r': 9,  'q': 10, 'k': 11,  # Black pieces
}
class ChessDataset(Dataset):
    def __init__(self, pgn_file, device='cpu'):
        self.device = device
        self.moves_data = []
        
        # 读取PGN文件中的游戏并生成每一步的p, q, r
        with open(pgn_file) as f:
            while game:=chess.pgn.read_game(f):
                moves = list(game.mainline_moves())
                board = game.board()
                for move in moves[:-1]:
                    p = self.board2vec(board)
                    legal_moves = list(board.legal_moves)
                    pseudo_move = random.choice(legal_moves)
                    board.push(pseudo_move)
                    r = self.board2vec(board)
                    board.pop()
                    board.push(move)
                    q = self.board2vec(board)
                    self.moves_data.append((p, q, r))

    def __len__(self):
        return len(self.moves_data)
    
    def board2vec(self, board):
        # 使用 NumPy 进行初始处理
        vec = np.zeros((12, 8, 8), dtype=np.float32)
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                piece_index = piece_to_index[piece.symbol()]
                row, col = divmod(square, 8)
                vec[piece_index, row, col] = 1
        
        # 将 NumPy 数组转换为 PyTorch 张量并移动到设备上
        vec = torch.tensor(vec, dtype=torch.float32).to(self.device)
        return vec
    
    def __getitem__(self, idx):
        return self.moves_data[idx]

class ChessValueNetwork(nn.Module):
    def __init__(self):
        super(ChessValueNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(12, 64, kernel_size=3, padding=1),  # [12, 8, 8] -> [64, 8, 8]
            nn.BatchNorm2d(64),  # Batch Normalization
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # [64, 8, 8] -> [128, 8, 8]
            nn.BatchNorm2d(128),  # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # [128, 8, 8] -> [128, 4, 4]
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # [128, 4, 4] -> [256, 4, 4]
            nn.BatchNorm2d(256),  # Batch Normalization
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)  # [256, 4, 4] -> [256, 2, 2]
        )
        self.fc = nn.Sequential(
            nn.Linear(256 * 2 * 2, 512),  # [1, 256*2*2] -> [1, 512]
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout
            nn.Linear(512, 256),  # [1, 512] -> [1, 256]
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout
            nn.Linear(256, 1),  # 输出标量
            nn.Tanh()  # 限制在 [-1, 1]
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)  # 展平成 [Batch Size, Features]
        x = self.fc(x)
        return x

def objective_function(model, p, q, r, kappa=10.0):
    # Forward pass: Compute scores for p, q, and r
    f_p = model(p).squeeze()  # Score for p
    f_q = model(q).squeeze()  # Score for q
    f_r = model(r).squeeze()  # Score for r
    # Loss components
    # Loss A: Ensure f(q) > f(r) (optimal move vs random move)
    loss_a = -torch.log(F.sigmoid(f_q - f_r)).mean()
    # Loss B: Ensure f(p) + f(q) close to zero (soft equality constraint)
    loss_b = -torch.log(F.sigmoid(kappa * (f_p + f_q))).mean()
    # Loss C: Ensure -f(p) - f(q) close to zero (soft equality constraint)
    loss_c = -torch.log(F.sigmoid(-kappa * (f_p + f_q))).mean()
    # Total loss: Combine all components
    total_loss = loss_a + loss_b + loss_c

    return total_loss

def lr_lambda(current_epoch):
    current_time = time.time()  # 当前时间戳
    elapsed_time = current_time - t0
    return math.exp(-elapsed_time / 86400)

model = ChessValueNetwork().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.03, weight_decay=LAMBDA)
t0 = time.time()
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
writer = SummaryWriter(log_dir="./logs")

In [3]:
pgn_file = "./data/chessGame0.pgn"
print("creating dataset for " + pgn_file)
chess_dataset = ChessDataset(pgn_file, device=device)
dataloader = DataLoader(chess_dataset, batch_size=1, shuffle=True)

creating dataset for ./data/chessGame0.pgn


In [ ]:
def board2vec(self, board):
        # 使用 NumPy 进行初始处理
        vec = np.zeros((12, 8, 8), dtype=np.float32)
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                piece_index = piece_to_index[piece.symbol()]
                row, col = divmod(square, 8)
                vec[piece_index, row, col] = 1
        
        # 将 NumPy 数组转换为 PyTorch 张量并移动到设备上
        vec = torch.tensor(vec, dtype=torch.float32).to(self.device)
        return vec
# checkmate 1-0
strio = 
board = chess.Board()


In [10]:
total_loss = 0
model.train()
for batch_idx, (p, q, r) in enumerate(dataloader):
    p, q, r = p.to(device), q.to(device), r.to(device)
    optimizer.zero_grad()
    loss = objective_function(model, p, q, r, KAPPA)
    loss.backward()
    # 梯度裁剪
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    scheduler.step()
    total_loss += loss.item()
    
    if batch_idx % 1000 == 0:
        print(f"PGN File [{pgn_file}], Batch [{batch_idx}], Loss: {loss.item():.4f}, lr: {scheduler.get_last_lr()[0]}")
        print(model(p))

PGN File [./data/chessGame0.pgn], Batch [0], Loss: 1.9773, lr: 0.02997369765696147
tensor([[0.1016]], device='cuda:0', grad_fn=<TanhBackward0>)
PGN File [./data/chessGame0.pgn], Batch [1000], Loss: 2.0794, lr: 0.02997123070268871
tensor([[-1.3438e-25]], device='cuda:0', grad_fn=<TanhBackward0>)
PGN File [./data/chessGame0.pgn], Batch [2000], Loss: 2.0794, lr: 0.02996876506862384
tensor([[-0.0008]], device='cuda:0', grad_fn=<TanhBackward0>)
PGN File [./data/chessGame0.pgn], Batch [3000], Loss: 2.0794, lr: 0.029966291939176437
tensor([[-0.0005]], device='cuda:0', grad_fn=<TanhBackward0>)
PGN File [./data/chessGame0.pgn], Batch [4000], Loss: 2.0794, lr: 0.029963831863886736
tensor([[-0.0001]], device='cuda:0', grad_fn=<TanhBackward0>)
PGN File [./data/chessGame0.pgn], Batch [5000], Loss: 2.0794, lr: 0.029961365821482466
tensor([[-0.0014]], device='cuda:0', grad_fn=<TanhBackward0>)
PGN File [./data/chessGame0.pgn], Batch [6000], Loss: 2.0794, lr: 0.029958903978094475
tensor([[0.0001]], dev

In [4]:
# 训练代码
def train_model(model, optimizer, scheduler, objective_function, pgn_files, device, EPOCHS, KAPPA, writer):
    for epoch in range(EPOCHS):
        total_loss = 0
        model.train()
        
        for pgn_file in pgn_files:
            # 为每个PGN文件创建数据集和数据加载器
            print("creating dataset for " + pgn_file)
            chess_dataset = ChessDataset(pgn_file, device=device)
            dataloader = DataLoader(chess_dataset, batch_size=1, shuffle=True)
            
            for batch_idx, (p, q, r) in enumerate(dataloader):
                p, q, r = p.to(device), q.to(device), r.to(device)
                optimizer.zero_grad()
                loss = objective_function(model, p, q, r, KAPPA)
                loss.backward()
                # 梯度裁剪
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                scheduler.step()
                total_loss += loss.item()
                
                if batch_idx % 1000 == 0:
                    print(f"Epoch [{epoch+1}/{EPOCHS}], PGN File [{pgn_file}], Batch [{batch_idx}], Loss: {loss.item():.4f}, lr: {scheduler.get_last_lr()[0]}")
            writer.add_text("PGN Files", str(pgn_files), epoch)
        
        # 记录损失,学习率,PGNfile到 TensorBoard
        writer.add_scalar("Loss/train", total_loss, epoch)
        writer.add_scalar("Learning Rate", scheduler.get_last_lr()[0], epoch)
        print(f"Epoch [{epoch+1}/{EPOCHS}], Total Loss: {total_loss:.4f}")

    writer.close()
pgn_files = [f"./data/chessGame{i}.pgn" for i in range(4812)]
train_model(model, optimizer, scheduler, objective_function, pgn_files, device, EPOCHS, KAPPA, writer)


creating dataset for ./data/chessGame0.pgn
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [0], Loss: 2.3184, lr: 0.029986162701996198
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [1000], Loss: 2.0794, lr: 0.02998372358738248
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [2000], Loss: 2.0794, lr: 0.029981299299038686
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [3000], Loss: 2.0795, lr: 0.02997886849349813
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [4000], Loss: 2.0795, lr: 0.029976446743442426
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [5000], Loss: 2.0795, lr: 0.029974031373603882
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [6000], Loss: 2.0794, lr: 0.0299715957369514
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [7000], Loss: 2.0795, lr: 0.02996917323707462
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [8000], Loss: 2.0794, lr: 0.029966716951123983
Epoch [1/10], PGN File [./data/chessGame0.pgn], Batch [9

KeyboardInterrupt: 

In [ ]:
model.save("1.pth")